Submitting various things for end of grant.

In [ ]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [ ]:
from curation_common import *


In [ ]:
GCATDIR = os.path.expanduser('~/src/gcat')
if GCATDIR not in sys.path:
    sys.path.append(GCATDIR)
    import gcat

In [ ]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)

In [ ]:
from encoded_client.encoded import DCCValidator
from encoded_client.encoded import Document

In [ ]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = "ENCODE_miRNA_experiments_06122019"
# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [ ]:
award = 'UM1HG009443'

# Submit Documents

Example Document submission

In [ ]:
#atac_uuid = '0fc44318-b802-474e-8199-f3b6d708eb6f'
#atac = Document(os.path.expanduser('~/proj/encode3-curation/Wold_Lab_ATAC_Seq_protocol_December_2016.pdf'),
#                'general protocol',
#                'ATAC-Seq experiment protocol for Wold lab',
#                )
#body = atac.create_if_needed(server, atac_uuid)
#print(body['@id'])

# Submit Annotations

In [ ]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [ ]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [ ]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosample = book.parse('Biosamples', header=0)
for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and pandas.isnull(row.source):
        obj = server.get_json(row.accession)
        print(obj['source']['@id'])

In [ ]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosample = book.parse('Biosamples', header=0)
created = server.post_sheet('/biosamples/', 
                            biosample,
                            verbose=True,
                            dry_run=True,
                            validator=validator)
print(len(created))

In [ ]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [ ]:
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = book.parse('Libraries', header=0)
created = server.post_sheet('/libraries/',
                            libraries,
                            verbose=True,
                            dry_run=True,
                            validator=validator)
print(len(created))

In [ ]:
#if created:
libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [ ]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = book.parse('Experiments', header=0)
for i, row in experiments.iterrows():
    if not pandas.isnull(row.accession) and pandas.isnull(row.biosample_ontology):
        obj = server.get_json(row.accession)
        print(obj['biosample_ontology']['@id'])

In [ ]:
print(server.server)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = book.parse('Experiments', header=0)
created = server.post_sheet('/experiments/',
                            experiments,
                            verbose=True,
                            dry_run=True,
                            validator=validator)
print(len(created))

In [ ]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [ ]:
print(server.server)
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = book.parse('Replicates', header=0)
created = server.post_sheet('/replicates/', 
                            replicates,
                            verbose=True,
                            dry_run=True,
                            validator=validator)
print(len(created))

In [ ]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

In [ ]:
with pandas.ExcelWriter('/dev/shm/submission.xlsx') as book:
    biosample.to_excel(book, 'Biosample', index=False)
    experiments.to_excel(book, 'Experiment', index=False)
    libraries.to_excel(book, 'Library', index=False)
    replicates.to_excel(book, 'Replicate', index=False)
